# Installation

In [ ]:
# pip install gensim requests rouge bert_score openpyxl prettytable nltk gdown

In [ ]:
# import nltk
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('punkt_tab')

In [2]:
# import gdown
# import gzip
# import shutil
# import os

# compressed_path = "./GoogleNews-vectors-negative300.bin.gz"
# decompressed_path = "./GoogleNews-vectors-negative300.bin"


# # Google Drive file ID extracted from the link
# file_id = "0B7XkCwpI5KDYNlNUTTlSS21pQmM"

# # Download the file
# gdown.download(f"https://drive.google.com/uc?id={file_id}&export=download", compressed_path, quiet=False)

# print("Download completed.")


# # Extract the .bin file from .gz
# with gzip.open(compressed_path, "rb") as f_in:
#     with open(decompressed_path, "wb") as f_out:
#         shutil.copyfileobj(f_in, f_out)

# print("Extraction completed.")

# # (Optional) Delete the .gz file after extraction
# os.remove(compressed_path)
# print("Deleted the compressed file.")

# Performance Evaluation

In [1]:
import pandas as pd
import warnings
import logging

warnings.filterwarnings("ignore")
logging.getLogger('transformers').setLevel(logging.ERROR)

# Set the logging level to ERROR to ignore warnings
logging.getLogger("transformers").setLevel(logging.ERROR)


In [2]:
# DATASET = "Blended Skill Talk"                                             
DATASET = "IT-ConvAI2"                                             
# DATASET = "FoCus"                                             

# LLM = "Qwen2-5B-Benchmark"     
# LLM = "Qwen2-5B-DPO"    
# LLM = "Qwen2-5B-DPO-AVG"     
# LLM = "Qwen2-5B-DPO-LENGTH-PRIOR"   


LLM = "Qwen2-7B-Instruct"
# LLM = "Mistral-7B-Instruct"
# LLM = "Llama3-1-8B-Instruct"
                              
COT_SETUP = False

In [3]:
df = pd.read_csv(f'./Prompts/{DATASET}.csv')
print("Shape:", df.shape)

df.head(2)

Shape: (1183, 3)


,personas,context,act_response
0,i love disneyland and mickey mouse.i love to s...,"User1: no , we recently purchased a new house ...",User2: yes i love mickey mouse such a cute lit...
1,i love to drink fancy tea.i have a big library...,User1: hi how are you doing ? i am okay how ab...,User2: i am doing good . just sipping tea . wh...


In [4]:
### Only For: FoCus, IT-ConvAI2

df['act_response'] = df['act_response'].apply(lambda x: x.split(':', 1)[1].strip() if ':' in x else x.strip())

print(df.isnull().sum())
df.head(2)

personas        0
context         0
act_response    0
dtype: int64


,personas,context,act_response
0,i love disneyland and mickey mouse.i love to s...,"User1: no , we recently purchased a new house ...",yes i love mickey mouse such a cute little rat
1,i love to drink fancy tea.i have a big library...,User1: hi how are you doing ? i am okay how ab...,i am doing good . just sipping tea . what do y...


In [5]:
COT_ = "-COT" if COT_SETUP else ""

response = pd.read_csv(f'Responses/{DATASET}/{LLM}{COT_}.csv')
print("Shape:", response.shape)

print("\nMissing Values:")
print(response.isnull().sum())

response

Shape: (1183, 2)

Missing Values:
gen_response     22
response_time     0
dtype: int64


,gen_response,response_time
0,I'm sorry to hear about your new house purchas...,1.600242
1,"I'm doing well, thank you for asking. I'm just...",1.664674
2,"My name is Cloudy, and I'm a little girl with ...",1.387386
3,"Yes, I love turtles! They're so cute and fasci...",1.314589
4,My cats' names are Whiskers and Mittens. They'...,1.070947
...,...,...
1178,"Oh, interesting! I'm actually a musician. I cr...",1.278092
1179,Scorpions are actually neither insects nor ani...,3.845814
1180,"I work as a custodian to help pay the bills, b...",1.871604
1181,"I'm a vegan who loves hummus, roller coasters,...",2.051421


In [6]:
# Calculate maximum number of words in each column
max_response_length = response['gen_response'].dropna().apply(lambda x: len(x.split())).max()

print(f"Maximum Response Length (in words): {max_response_length}")

Maximum Response Length (in words): 88


In [7]:
import pandas as pd
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Initialize stop words
stop_words = set(stopwords.words('english'))

# Function to preprocess text
def preprocess_text(text, remove_stop_words=True):
    if pd.isnull(text):
        return None
    text = text.lower()  # Lowercasing
    text = text.translate(str.maketrans('', '', string.punctuation))  # Removing punctuation
    tokens = word_tokenize(text)  # Tokenization
    if remove_stop_words:
        tokens = [word for word in tokens if word not in stop_words]  # Removing stop words
    return ' '.join(tokens)  # Join tokens back into a single string


# Create eval_df
eval_df = pd.DataFrame({
    'personas': df['personas'],
    'context': df['context'],
    'act_response': df['act_response'],
    'gen_response': response['gen_response'],
    'response_time': response['response_time']
})

print(eval_df.isnull().sum())
eval_df.head()

personas          0
context           0
act_response      0
gen_response     22
response_time     0
dtype: int64


,personas,context,act_response,gen_response,response_time
0,i love disneyland and mickey mouse.i love to s...,"User1: no , we recently purchased a new house ...",yes i love mickey mouse such a cute little rat,I'm sorry to hear about your new house purchas...,1.600242
1,i love to drink fancy tea.i have a big library...,User1: hi how are you doing ? i am okay how ab...,i am doing good . just sipping tea . what do y...,"I'm doing well, thank you for asking. I'm just...",1.664674
2,im a little girl.ive superpowers.i like to mak...,User1: what is your name ? are you a male or f...,im a girl . i do not give out my name .,"My name is Cloudy, and I'm a little girl with ...",1.387386
3,i love cats and have two cats.my favorite seas...,User1: hi ! do you like turtles ?,i am much more of a cat person actually,"Yes, I love turtles! They're so cute and fasci...",1.314589
4,i love cats and have two cats.my favorite seas...,User1: what are your kitties names ?,"snow and winter , named after my favorite season",My cats' names are Whiskers and Mittens. They'...,1.070947


In [8]:
import torch
device = 0 if torch.cuda.is_available() else -1  # device set to 0 for GPU, -1 for CPU

In [9]:
from transformers import pipeline, BertForSequenceClassification, BertTokenizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from transformers import pipeline
from tqdm import tqdm
import pandas as pd
import numpy as np
import gensim
import sys
import os


# Add UniEval to PYTHONPATH
sys.path.append(os.path.abspath("UniEval"))  # Update with your actual path

from UniEval.utils import convert_to_json
from UniEval.metric.evaluator import get_evaluator


# Lists to store the metrics
ue_scores = []
c_scores = []
persona_distance_scores = []
coh_unieval_scores = []


bert_snli_dir = "Fine-tuning/output/bert_snli"
bert_snli_model = BertForSequenceClassification.from_pretrained(bert_snli_dir)
bert_snli_tokenizer = BertTokenizer.from_pretrained(bert_snli_dir)

# Initialize the NLI pipeline for UE Score
bert_on_snli = pipeline('text-classification', model = bert_snli_model, tokenizer = bert_snli_tokenizer, device=0)

bert_dnli_dir = "Fine-tuning/output/bert_dnli"
bert_dnli_model = BertForSequenceClassification.from_pretrained(bert_dnli_dir)
bert_dnli_tokenizer = BertTokenizer.from_pretrained(bert_dnli_dir)

# Initialize the NLI pipeline
bert_on_dnli = pipeline('text-classification', model = bert_dnli_model, tokenizer = bert_dnli_tokenizer, device=0)


# Initialize the Word2Vec Model
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format("./GoogleNews-vectors-negative300.bin", binary=True)
stop_words = set(stopwords.words('english'))



def batch_calculate_c_scores(gen_responses, personas):
    """
    Batched version of calculate_c_score using DNLI model.
    """
    assert len(gen_responses) == len(personas), "Mismatched input lengths"

    inputs = [f"{p} {r}" for p, r in zip(personas, gen_responses)]
    results = bert_on_dnli(inputs)

    label_mapping = {
        'LABEL_0': 'negative',
        'LABEL_1': 'neutral',
        'LABEL_2': 'positive'
    }

    scores = []
    for result in results:
        label = label_mapping.get(result['label'], 'unknown')
        if label == 'positive':
            scores.append(1)
        elif label == 'neutral':
            scores.append(0)
        elif label == 'negative':
            scores.append(-1)
        else:
            scores.append(None)
    return scores


# def calculate_c_score(gen_response, persona):
#     """
#     Calculate the C score based on the entailment results between a generated response (R)
#     and a given persona (P).

#     Returns:
#     int: C-score with possible values:
#          1 for entailment (positive),
#          0 for neutral,
#          -1 for contradiction (negative).
#     """

#     # Define the label mapping to interpret the NLI model's output
#     label_mapping = {
#         'LABEL_0': 'negative',
#         'LABEL_1': 'neutral',
#         'LABEL_2': 'positive'
#     }
    
#     # Check entailment between persona (P) and generated response (R)
#     result_pr = bert_on_dnli(f"{persona} {gen_response}")
#     label_pr = label_mapping.get(result_pr[0]['label'], 'unknown')

#     # Determine C score based on entailment results
#     if label_pr == 'positive':
#         return 1
#     elif label_pr == 'neutral':
#         return 0
#     elif label_pr == 'negative':
#         return -1
#     else:
#         raise ValueError(f"Unexpected label encountered: {label_pr}")


# def calculate_ue_score(act_response, gen_response, persona):
#     """
#     Calculate the UE score based on entailment between persona, actual response, and generated response.

#     Returns:
#     int: UE score with possible values 2, 1, or 0.
#     """

#     # Define the label mapping to interpret the NLI model's output
#     label_mapping = {
#         'LABEL_0': 'entailment',
#         'LABEL_1': 'neutral',
#         'LABEL_2': 'contradiction'
#     }
    
#     # Check entailment between persona (P) and generated response (R)
#     result_pr = bert_on_snli(f"{persona} [SEP] {gen_response}")
#     label_pr = label_mapping.get(result_pr[0]['label'], 'unknown')

#     # Check entailment between actual response (Q) and generated response (R)
#     result_qr = bert_on_snli(f"{act_response} [SEP] {gen_response}")
#     label_qr = label_mapping.get(result_qr[0]['label'], 'unknown')

#     # Determine UE score based on entailment results
#     if label_pr == 'entailment' and label_qr == 'entailment':
#         return 2
#     elif label_pr == 'entailment':
#         return 1
#     else:
#         return 0


def batch_calculate_ue_scores(act_responses, gen_responses, personas):
    """
    Batched version of calculate_ue_score using SNLI model.
    Returns a list of UE scores (0, 1, or 2).
    """
    assert len(act_responses) == len(gen_responses) == len(personas), "Mismatched lengths."

    # Prepare NLI inputs
    inputs_pr = [f"{p} [SEP] {r}" for p, r in zip(personas, gen_responses)]
    inputs_qr = [f"{q} [SEP] {r}" for q, r in zip(act_responses, gen_responses)]

    # Run both batches
    results_pr = bert_on_snli(inputs_pr)
    results_qr = bert_on_snli(inputs_qr)

    label_mapping = {
        'LABEL_0': 'entailment',
        'LABEL_1': 'neutral',
        'LABEL_2': 'contradiction'
    }

    scores = []
    for res_pr, res_qr in zip(results_pr, results_qr):
        label_pr = label_mapping.get(res_pr['label'], 'unknown')
        label_qr = label_mapping.get(res_qr['label'], 'unknown')

        if label_pr == 'entailment' and label_qr == 'entailment':
            scores.append(2)
        elif label_pr == 'entailment':
            scores.append(1)
        else:
            scores.append(0)
    return scores



# def calculate_coh_unieval_score(personas, contexts, gen_responses):
#     """
#     Args:
#         personas (list): List of persona information as additional context.
#         contexts (str or list): Conversation histories leading to the responses.
#         gen_responses (str or list): Generated responses to be evaluated.

#     Returns:
#         float: The coherence score.
#     """
    
#     # Ensure personas is a list and flatten if necessary
#     personas = [' '.join(p) if isinstance(p, list) else p for p in personas]

#     # Ensure contexts and gen_responses are lists
#     if isinstance(contexts, str):
#         contexts = [contexts]  # Convert single string to list

#     if isinstance(gen_responses, str):
#         gen_responses = [gen_responses]  # Convert single string to list

#     # Prepare inputs for UniEval
#     data = convert_to_json(output_list=gen_responses, src_list=contexts, context_list=personas)

#     # Initialize the evaluator for dialogue tasks
#     evaluator = get_evaluator('dialogue')

#     # Evaluate and obtain scores for all inputs
#     eval_scores = evaluator.evaluate(data, print_result=False)
    
#     # Extract and return only the first coherence score
#     return eval_scores[0].get("coherence", None) if eval_scores else None


def batch_calculate_coh_unieval_scores(personas_list, contexts_list, gen_responses_list):
    """
    Batched coherence scoring using UniEval evaluator.
    """
    # Ensure personas are joined as single strings
    personas_list = [' '.join(p) if isinstance(p, list) else p for p in personas_list]
    
    # Convert to UniEval input format
    data = convert_to_json(
        output_list=gen_responses_list,
        src_list=contexts_list,
        context_list=personas_list
    )
    
    # Load UniEval dialogue evaluator only once
    evaluator = get_evaluator('dialogue')
    eval_scores = evaluator.evaluate(data, print_result=False)

    # Extract coherence scores
    return [s.get("coherence", None) for s in eval_scores]




def compute_persona_distance(persona, response, model, stop_words):
    # Tokenize and filter stopwords
    persona_tokens = [word for word in persona.lower().split() if word not in stop_words]
    response_tokens = [word for word in response.lower().split() if word not in stop_words]
    
    # Get word vectors
    persona_vecs = [model[word] for word in persona_tokens if word in model]
    response_vecs = [model[word] for word in response_tokens if word in model]
    
    # If no vectors found, return zero similarity
    if not persona_vecs or not response_vecs:
        return 0.0
    
    # Compute average vectors
    persona_avg_vec = np.mean(persona_vecs, axis=0)
    response_avg_vec = np.mean(response_vecs, axis=0)
    
    # Compute cosine similarity
    return cosine_similarity([persona_avg_vec], [response_avg_vec])[0][0]



Device set to use cuda:0
Device set to use cuda:0


In [10]:
import logging

# Set the logging level to ERROR to suppress warnings about training
logging.getLogger("transformers").setLevel(logging.ERROR)

# Default worst-case values
worst_c_score = -1.0
worst_ue_score = 0.0
worst_persona_distance_score = 0.0
worst_coh_unieval_score = 0.0

In [11]:
from tqdm import tqdm

# Identify valid rows (non-null gen_response)
valid_mask = ~eval_df['gen_response'].isna()
valid_indices = eval_df[valid_mask].index.tolist()

# Extract valid inputs
valid_personas = eval_df.loc[valid_indices, 'personas'].tolist()
valid_act_responses = eval_df.loc[valid_indices, 'act_response'].tolist()
valid_contexts = eval_df.loc[valid_indices, 'context'].tolist()
valid_gen_responses = eval_df.loc[valid_indices, 'gen_response'].tolist()

# === Compute batch metrics with tqdm logging ===
print("Calculating C Scores...")
c_scores_batch = batch_calculate_c_scores(valid_gen_responses, valid_personas)

print("Calculating UE Scores...")
ue_scores_batch = batch_calculate_ue_scores(valid_act_responses, valid_gen_responses, valid_personas)

print("Calculating UniEval Coherence Scores...")
coh_unieval_batch_scores = batch_calculate_coh_unieval_scores(valid_personas, valid_contexts, valid_gen_responses)

# Initialize all score lists with worst-case values
c_scores = [worst_c_score] * len(eval_df)
ue_scores = [worst_ue_score] * len(eval_df)
coh_unieval_scores = [worst_coh_unieval_score] * len(eval_df)

# Fill valid indices with batch results using progress bar
print("Filling final metric arrays...")
for i, c, ue, coh in tqdm(zip(valid_indices, c_scores_batch, ue_scores_batch, coh_unieval_batch_scores),
                          total=len(valid_indices), desc="Populating Scores"):
    c_scores[i] = c
    ue_scores[i] = ue
    coh_unieval_scores[i] = coh

Calculating C Scores...
Calculating UE Scores...
Calculating UniEval Coherence Scores...
Filling final metric arrays...


Populating Scores: 100%|██████████| 1161/1161 [00:00<00:00, 3164124.07it/s]


In [12]:
# Iterate over each row
for index, row in tqdm(eval_df.iterrows(), total=len(eval_df)):
    personas = row['personas']
    contexts = row['context']
    act_response = row['act_response']
    gen_response = row['gen_response']
    
    # Check for NaN or None in gen_response
    if pd.isna(gen_response):
    
        persona_distance_scores.append(worst_persona_distance_score)
        
        continue

    persona_distance_scores.append(compute_persona_distance(personas, gen_response, word2vec_model,stop_words))


# Compile metrics into DataFrame
metrics_df = pd.DataFrame({
    'Coh-UniEval': coh_unieval_scores,
    'C Score': c_scores,
    'UE Score': ue_scores,
    'Persona Distance': persona_distance_scores
})

metrics_df

100%|██████████| 1183/1183 [00:00<00:00, 3432.06it/s]


,Coh-UniEval,C Score,UE Score,Persona Distance
0,0.984436,0.0,0.0,0.586262
1,0.989371,0.0,0.0,0.514717
2,0.284176,1.0,2.0,0.708235
3,0.997843,0.0,0.0,0.637448
4,0.995969,1.0,0.0,0.419571
...,...,...,...,...
1178,0.994208,-1.0,0.0,0.611310
1179,0.993923,1.0,2.0,0.584755
1180,0.997008,1.0,0.0,0.845741
1181,0.999340,0.0,1.0,0.769901


In [13]:
# # Storing the full results
output_path = f'./Metrics Results/{DATASET}/{LLM}{COT_}-results.xlsx'

df_concat = pd.concat([eval_df, metrics_df], axis=1)

df_concat.to_excel(output_path, index=False)

# Load and Aggregate Results

In [14]:
import pandas as pd

df = pd.read_excel(f"Metrics Results/{DATASET}/{LLM}-results.xlsx")
metrics_df = df.drop(columns=["personas", "context", "act_response","gen_response"])
metrics_df

,response_time,Coh-UniEval,C Score,UE Score,Persona Distance
0,1.600242,0.984436,0,0,0.586262
1,1.664674,0.989371,0,0,0.514717
2,1.387386,0.284176,1,2,0.708235
3,1.314589,0.997843,0,0,0.637448
4,1.070947,0.995969,1,0,0.419571
...,...,...,...,...,...
1178,1.278092,0.994208,-1,0,0.611310
1179,3.845814,0.993923,1,2,0.584755
1180,1.871604,0.997008,1,0,0.845741
1181,2.051421,0.999340,0,1,0.769901


In [15]:
# Calculate the mean (average) and standard deviation, rounded to 2 decimal places
avg_values = metrics_df.mean().round(2)
std_values = metrics_df.std(ddof=0).round(2)  # Use ddof=0 for population standard deviation

# Combine the average and standard deviation into the format "avg ± std"
combined_values = avg_values.astype(str) + " ± " + std_values.astype(str)

# Insert the LLM name at the beginning of the combined values
combined_values = combined_values.tolist()
combined_values.insert(0, LLM)

# Create a DataFrame for the combined average ± std row
result_df = pd.DataFrame([combined_values], columns=['Model'] + metrics_df.columns.tolist())

# Add the ratio of invalid gen_response
invalid_gen_res_ratio = df['gen_response'].isna().sum() /len(df) 

result_df['Failure Ratio'] = f"{round(invalid_gen_res_ratio, 3)} ± 0.00"  # No std for Failure Ratio

result_df

,Model,response_time,Coh-UniEval,C Score,UE Score,Persona Distance,Failure Ratio
0,Qwen2-7B-Instruct,1.97 ± 0.72,0.89 ± 0.24,0.23 ± 0.59,0.42 ± 0.73,0.61 ± 0.17,0.019 ± 0.00


In [16]:
# Load the existing Excel file and update or append the average row
output_path = f'./Evaluations/{DATASET}{COT_}-results.xlsx'

try:
    # Load existing data
    existing_df = pd.read_excel(output_path)
    # Check if the model name already exists
    if LLM in existing_df['Model'].values:
        # Update the row with the same model name
        existing_df.loc[existing_df['Model'] == LLM, :] = result_df.values
    else:
        # Append the new data
        existing_df = pd.concat([existing_df, result_df], ignore_index=True)
except FileNotFoundError:
    # If the file does not exist, create a new DataFrame
    existing_df = result_df

# Save the updated DataFrame to an Excel file
existing_df.to_excel(output_path, index=False)

existing_df

,Model,response_time,Coh-UniEval,C Score,UE Score,Persona Distance,Failure Ratio
0,Llama3-1-8B-Instruct,4.15 ± 0.53,0.82 ± 0.34,0.12 ± 0.66,0.23 ± 0.58,0.55 ± 0.22,0.098 ± 0.00
1,Mistral-7B-Instruct,3.11 ± 1.11,0.86 ± 0.28,0.36 ± 0.69,0.46 ± 0.72,0.62 ± 0.2,0.047 ± 0.00
2,Qwen2-7B-Instruct,1.97 ± 0.72,0.89 ± 0.24,0.23 ± 0.59,0.42 ± 0.73,0.61 ± 0.17,0.019 ± 0.00


## Reviwing the Results

In [17]:
# DATASET = "FoCus"  
# # COT_ = "-COT"
# COT_ =  ""

response = pd.read_excel(f'./Evaluations/{DATASET}{COT_}-results.xlsx')
response

,Model,response_time,Coh-UniEval,C Score,UE Score,Persona Distance,Failure Ratio
0,Llama3-1-8B-Instruct,4.15 ± 0.53,0.82 ± 0.34,0.12 ± 0.66,0.23 ± 0.58,0.55 ± 0.22,0.098 ± 0.00
1,Mistral-7B-Instruct,3.11 ± 1.11,0.86 ± 0.28,0.36 ± 0.69,0.46 ± 0.72,0.62 ± 0.2,0.047 ± 0.00
2,Qwen2-7B-Instruct,1.97 ± 0.72,0.89 ± 0.24,0.23 ± 0.59,0.42 ± 0.73,0.61 ± 0.17,0.019 ± 0.00
